In [1]:
from collections import Counter
import pickle
import subprocess

In [2]:
en_corpus=[]
es_corpus=[]
with open('corpus.en') as f1:
    while True:
        en = f1.readline()
        if en =='' or en is None:
            break
        en_corpus.append(['NULL']+[w for w in en.strip().split()])
        
with open('corpus.es') as f2:
    while True:
        es = f2.readline()
        if es=='' or es is None:
            break
        es_corpus.append([w for w in es.strip().split()])

In [ ]:
# en_vocab=Counter([w for s in en_corpus for w in s])
# es_vocab=Counter([w for s in es_corpus for w in s])

In [ ]:
# print(len([w for s in en_corpus for w in s]))
# print(len([w for s in es_corpus for w in s]))

In [3]:
print(len(en_corpus))
print(len(es_corpus))

5401
5401


In [9]:
with open('t_ibm1.pickle','rb') as fr:
    t=pickle.load(fr)
# t=Counter()
# for k in range(len(en_corpus)):
#     for e in en_corpus[k]:
#         for f in es_corpus[k]:
#             t[(f,e)]=1/en_vocab[e]

In [ ]:
t

In [5]:
n_iter=5

In [10]:
q=Counter()
for k in range(len(en_corpus)):
    l_en = len(en_corpus[k])
    m_es = len(es_corpus[k])
    for i in range(1,m_es+1):
        for j in range(l_en+1):
            q[(j,i,l_en,m_es)] = 1/(l_en+1)

In [ ]:
print(len(q))

In [12]:
for iterno in range(n_iter):
    print("Iteration #{}".format(iterno))
    c=Counter()
    for k in range(len(en_corpus)):
        for i in range(1,len(es_corpus[k])+1):
            sum1=0.0
            sumq=0.0
            sumt=0.0
            l_en = len(en_corpus[k])
            m_es = len(es_corpus[k])
            for j,w in enumerate(en_corpus[k]):
                sum1 += q[(j,i,l_en, m_es)] * t[(es_corpus[k][i-1],w)]
                sumt += t[(es_corpus[k][i-1],w)]
                sumq += q[(j,i,l_en, m_es)]
            if sum1 == 0:
                print(es_corpus[k],en_corpus[k])
                print(sumt,sumq)
            for j in range(len(en_corpus[k])):
                f = es_corpus[k][i-1]
                e = en_corpus[k][j]
                
                delta_kij= q[(j,i,l_en,m_es)] * t[(f,e)] / sum1
                c[(e,f)]+= delta_kij
                c[e] += delta_kij
                c[(j,i,len(en_corpus[k]),len(es_corpus[k]))] += delta_kij
                c[(i,len(en_corpus[k]),len(es_corpus[k]))] += delta_kij
#             if sum1!=0:
#                 delta_kij= t[(es_corpus[k][i],'NULL')] / sum1
#                 c[('NULL',es_corpus[k][i])]+= delta_kij
#                 c['NULL'] += delta_kij
#                 c[(-1,i,len(en_corpus[k]),len(es_corpus[k]))] += delta_kij
#                 c[(i,len(en_corpus[k]),len(es_corpus[k]))] += delta_kij
##############################################
#            Writing alignments 
##############################################
    for k in range(len(en_corpus)):
        for i,f in enumerate(es_corpus[k],1):
            for j,e in enumerate(en_corpus[k]):
                t[(f,e)]=c[(e,f)]/c[e]
                q[(j,i,len(en_corpus[k]),len(es_corpus[k]))] = c[(j,i,len(en_corpus[k]),len(es_corpus[k]))]/c[(i,len(en_corpus[k]),len(es_corpus[k]))]
    alignments=[]
    for k in range(len(dev_es_corpus)):
        a_row=[]
        for i,f in enumerate(dev_es_corpus[k],1):
            l_en = len(dev_en_corpus[k])
            m_es = len(dev_es_corpus[k])
            score = max([ (q[(j,i,l_en,m_es)]*t[f,dev_en_corpus[k][j]],j) for j in range(len(dev_en_corpus[k]))])
            a_row.append(score[1])
        alignments.append(a_row)
    
    fname = 'ibm2_1_iteration_'+str(iterno)+'.out'
    with open(fname,'w') as f1:
        for i in range(len(alignments)):
            for j in range(len(alignments[i])):
                if alignments[i][j]!=0:
                    s = str(i+1) + " " + str(alignments[i][j]) + " " + str(j+1) + "\n"
                    f1.write(s)
    returned_output = subprocess.check_output("python3 eval_alignment.py dev.key " + fname,shell=True)
    print(returned_output.decode('utf-8'))

Iteration #0
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.431        0.445        0.438

Iteration #1
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.436        0.451        0.443

Iteration #2
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.439        0.454        0.446

Iteration #3
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.438        0.453        0.445

Iteration #4
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.442        0.456        0.449



In [ ]:
t

In [ ]:
with open('t_ibm2.pickle', 'wb') as fp:
    pickle.dump(t, fp)
with open('q_ibm2.pickle', 'wb') as fp:
    pickle.dump(q, fp)

In [ ]:
with open('t_ibm2.pickle','rb') as fr:
    t1=pickle.load(fr)

In [ ]:
t1

In [8]:
dev_en_corpus=[]
dev_es_corpus=[]
with open('dev.en') as f1:
    while True:
        en = f1.readline()
        if en =='' or en is None:
            break
        dev_en_corpus.append(['NULL']+[w for w in en.strip().split()])
        
with open('dev.es') as f2:
    while True:
        es = f2.readline()
        if es=='' or es is None:
            break
        dev_es_corpus.append([w for w in es.strip().split()])

In [ ]:
alignments=[]
for k in range(len(dev_es_corpus)):
    a_row=[]
    for i,f in enumerate(dev_es_corpus[k],1):
        l_en = len(dev_en_corpus[k])
        m_es = len(dev_es_corpus[k])
        score = max([ (q[(j,i,l_en,m_es)]*t[f,dev_en_corpus[k][j]],j) for j in range(len(dev_en_corpus[k]))])
        a_row.append(score[1])
    alignments.append(a_row)

In [ ]:
print(alignments)

In [ ]:
with open('ibm2_1.out','w') as f1:
    for i in range(len(alignments)):
        for j in range(len(alignments[i])):
            if alignments[i][j]!=0:
                s = str(i+1) + " " + str(alignments[i][j]) + " " + str(j+1) + "\n"
                f1.write(s)

In [ ]:
!python3 eval_alignment.py dev.key ibm2_1.out 

In [13]:
ibm_model2_alignments=set()
devkey_alignments=set()
with open('ibm2_1.out','r') as f1:
    while True:
        s= f1.readline()
        if s=='' or s is None:
            break
        else:
            a = tuple([int(i) for i in s.strip().split()])
            ibm_model2_alignments.add(a)

with open('dev.key','r') as f1:
    while True:
        s= f1.readline()
        if s=='' or s is None:
            break
        else:
            a = tuple([int(i) for i in s.strip().split()])
            devkey_alignments.add(a)

In [17]:
sents_all_correct=set()
sents_matches=set()
# sents_my_model_has_extra =set()
# sents_devkey_has_extra =set()

for k in range(1,len(dev_en_corpus)):
    m = len(dev_en_corpus[k-1])-1
    l = len(dev_es_corpus[k-1])
    all_correct=False
#     my_model_has_extra=True
#     devkey_has_extra=True
    count=0
    count_match=0
    devkey_set=set()
    ibm2_set=set()
    for j in range(1,m+1):
        for i in range(1,l+1):
            if (k,j,i) in devkey_alignments:
                devkey_set.add((k,j,i))
#                 all_correct=True
                count+=1
                if (k,j,i) in ibm_model2_alignments:
                    ibm2_set.add((k,j,i))
                    count_match+=1
#                     all_correct=False
#             if not((k,j,i) in devkey_alignments and (k,j,i) in ibm_model2_alignments):
#                 all_correct=False
#             if (k,j,i) in devkey_alignments and (k,j,i) not in ibm_model2_alignments:
#                 my_model_has_extra=False
#             if (k,j,i) not in devkey_alignments and (k,j,i) in ibm_model2_alignments:
#                 devkey_has_extra=False
    if count == count_match:
        sents_all_correct.add((k,tuple(dev_en_corpus[k-1]),tuple(dev_es_corpus[k-1]),tuple(sorted(list(devkey_set))),tuple(sorted(list(ibm2_set)))))
#     elif my_model_has_extra:
#         sents_my_model_has_extra.add(k)
#     elif devkey_has_extra:
#         sents_devkey_has_extra.add(k)

In [18]:
print("All correct: Sentences #{}".format(sents_all_correct))
# print("My model has extra alignments: Sentences #{}".format(sents_my_model_has_extra))
# print("Devkey has extra alignments: Sentences #{}".format(sents_devkey_has_extra))

All correct: Sentences #{(196, ('NULL', 'it', 'was', 'an', 'animated', ',', 'very', 'convivial', 'game', '.'), ('jugaban', 'de', 'una', 'manera', 'animada', 'y', 'muy', 'cordial', '.'), ((196, 4, 5), (196, 6, 7), (196, 7, 8), (196, 9, 9)), ((196, 4, 5), (196, 6, 7), (196, 7, 8), (196, 9, 9))), (192, ('NULL', 'one', 'issue', 'that', 'separates', 'us', 'is', 'the', 'civil', 'war', 'in', 'chechnya', '.'), ('una', 'cuestión', 'que', 'nos', 'separa', 'es', 'la', 'guerra', 'civil', 'en', 'chechenia', '.'), ((192, 1, 1), (192, 2, 2), (192, 3, 3), (192, 4, 5), (192, 5, 4), (192, 6, 6), (192, 7, 7), (192, 8, 9), (192, 9, 8), (192, 10, 10), (192, 11, 11), (192, 12, 12)), ((192, 1, 1), (192, 2, 2), (192, 3, 3), (192, 4, 5), (192, 5, 4), (192, 6, 6), (192, 7, 7), (192, 8, 9), (192, 9, 8), (192, 10, 10), (192, 11, 11), (192, 12, 12)))}


In [22]:
for k in [1,2]:
    m = len(dev_en_corpus[k-1])-1
    l = len(dev_es_corpus[k-1])
    devkey_set=set()
    ibm2_set=set()
    for j in range(1,m+1):
        for i in range(1,l+1):
            if (k,j,i) in devkey_alignments:
                devkey_set.add((k,j,i))
            if (k,j,i) in ibm_model2_alignments:
                ibm2_set.add((k,j,i))
    print(dev_en_corpus[k-1])
    print(dev_es_corpus[k-1])
    print(sorted(list(devkey_set)))
    print(sorted(list(ibm2_set)))

['NULL', 'my', 'group', 'can', 'see', 'no', 'reason', 'not', 'to', 'grant', 'discharge', 'for', 'the', '1996', 'budget', '.']
['mi', 'grupo', 'no', 've', 'razón', 'alguna', 'para', 'no', 'conceder', 'la', 'aprobación', 'de', 'la', 'gestión', 'correspondiente', 'al', 'presupuesto', 'de', '1996', '.']
[(1, 1, 1), (1, 2, 2), (1, 4, 4), (1, 5, 3), (1, 6, 5), (1, 7, 8), (1, 8, 7), (1, 9, 9), (1, 10, 11), (1, 12, 16), (1, 13, 19), (1, 14, 17), (1, 15, 20)]
[(1, 1, 1), (1, 2, 2), (1, 5, 8), (1, 6, 5), (1, 7, 3), (1, 9, 6), (1, 9, 9), (1, 9, 10), (1, 9, 15), (1, 9, 16), (1, 10, 11), (1, 10, 14), (1, 11, 7), (1, 11, 12), (1, 12, 13), (1, 13, 4), (1, 13, 18), (1, 13, 19), (1, 14, 17), (1, 15, 20)]
['NULL', 'i', 'hope', 'that', 'the', 'new', 'parliament', 'and', 'the', 'new', 'commission', 'will', 'make', 'use', 'of', 'them', '.']
['espero', 'que', 'el', 'nuevo', 'parlamento', 'y', 'la', 'nueva', 'comisión', 'puedan', 'hacer', 'uso', 'de', 'ellas', '.']
[(2, 2, 1), (2, 3, 2), (2, 4, 3), (2, 5, 4)

['NULL', 'my', 'group', 'can', 'see', 'no', 'reason', 'not', 'to', 'grant', 'discharge', 'for', 'the', '1996', 'budget', '.']
['mi', 'grupo', 'no', 've', 'razón', 'alguna', 'para', 'no', 'conceder', 'la', 'aprobación', 'de', 'la', 'gestión', 'correspondiente', 'al', 'presupuesto', 'de', '1996', '.']
{(1, 6, 5), (1, 15, 20), (1, 5, 3), (1, 8, 7), (1, 2, 2), (1, 4, 4), (1, 13, 19), (1, 10, 11), (1, 14, 17), (1, 7, 8), (1, 9, 9), (1, 1, 1), (1, 12, 16)}
{(1, 15, 20), (1, 9, 15), (1, 11, 12), (1, 12, 13), (1, 14, 17), (1, 9, 16), (1, 7, 3), (1, 13, 4), (1, 11, 7), (1, 13, 19), (1, 6, 5), (1, 13, 18), (1, 5, 8), (1, 2, 2), (1, 10, 11), (1, 9, 6), (1, 9, 9), (1, 1, 1), (1, 10, 14), (1, 9, 10)}
